In [3]:
import numpy as np
import sklearn
import tensorflow as tf
import pandas as pd

from keras.src.layers import Dense
from tensorflow import keras
from keras import layers, models
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential


# Load The Data

In [5]:
users = pd.read_csv("data/ratings.csv")

# Collaborative Filtering Model

### When you think collaborative filtering, think of statements like:
- Users who liked similar items also liked...
- Items similar to this item

### 🧠 “Behavioral Similarity”
The system learns from what users did, not what items are about.

> It doesn’t care what genre the item is — it just learns from the pattern of user behavior.

### 🔍 How It Works:
- Looks at user-item interactions (ratings, likes, views)
- Learns latent similarities between users or items
- Powered by embeddings, matrix factorization, or neural models

finds patterns in behavior



In [ ]:
num_users = users['user_id'].nunique()
num_items = users['item_id'].nunique()
embedding_dim = 32
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)

# Content-Based Filtering Model

When you think of content-based filtering, think of statements like:
- Because you liked horror
- Because you searched laptops

### 🧠 “Attribute Similarity”
The system uses the metadata or features of items (or users) directly.

> It recommends items with similar features to what you liked, not because other users liked them.

### 📦 How It Works:
- Uses item (or user) attributes: genres, categories, descriptions
- Builds a user profile from liked item features
- Compares feature vectors (e.g., via cosine similarity)

finds patterns in features

Collaborative filtering learns from who likes what, no matter what it is.

Content-based filtering learns from what the thing is, no matter who liked it.

In [ ]:
user_NN = Sequential([
    layers.Input(shape = num_user_features),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32)
])

item_NN = Sequential([
    layers.Input(shape= num_item_features),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32)
])

vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = layers.Dot(axes=1)([vu, vm])

model = Model([input_user, input_item], output)

cost_fn = keras.losses.MeanSquaredError()

# Training

# Testing